In [1]:
from client import Client
from server import Server
from validation_tools import load_sampled_dataset, compute_prf
from evaluate import load as load_metric
import pandas as pd
from datasets import load_dataset

# DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

/home/weiwei/miniconda3/envs/rag_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
common_sense = Client(vectorstore_path="./common_sense_db")
common_sense.build_vectorstore(folder_path='./classified_dataset/common_sense')

In [ ]:
computer_science_coding = Client(vectorstore_path="./computer_science_coding_db")
computer_science_coding.build_vectorstore(folder_path='./classified_dataset/computer_science_coding_related')

In [2]:
law = Client(vectorstore_path="./law_related_db")
law.build_vectorstore(folder_path='./classified_dataset/law_related')

/home/weiwei/RAGnet/client.py:60: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


Total chunks after Character split: 1
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after Character split: 1
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after Character split: 2
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after Character split: 5
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 1
Inserted batch up to merged chunk 1/1
Total chunks after Character split: 1
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after Character split: 2
Total chunks after sem

In [2]:
medicine = Client(vectorstore_path="./medicine_related_db")
medicine.build_vectorstore(folder_path='./classified_dataset/medicine_related')

/home/weiwei/RAGnet/client.py:60: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


Total chunks after Character split: 2
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after Character split: 16
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 2
Inserted batch up to merged chunk 2/2
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after semantic split: 3
Inserted batch up to merged chunk 3/3
Total chunks after sema